In [242]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [243]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [244]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [245]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [246]:
STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  

STARTINGCSV=pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
8254,2023020001,NSH,R,2023-10-10,0,0,{'default': 'Predators'},{'default': 'Lightning'},0,-2,0,0,0,0,2,25,0,0,TBL,4,25:37,8474600,Roman,Josi
4362,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},0,0,0,0,0,0,0,19,0,0,SEA,2,16:09,8481167,Brayden,Pachal
10705,2023020002,PIT,H,2023-10-10,0,0,{'default': 'Penguins'},{'default': 'Blackhawks'},0,0,0,0,0,0,1,17,0,0,CHI,2,10:14,8470604,Jeff,Carter
18850,2023020002,CHI,R,2023-10-10,0,0,{'default': 'Blackhawks'},{'default': 'Penguins'},0,-1,0,0,0,0,0,22,0,0,PIT,0,17:58,8476473,Connor,Murphy
3898,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},0,1,0,0,0,0,1,18,0,0,SEA,0,15:12,8476448,William,Karlsson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2023020670,TOR,H,2024-01-14,0,0,{'default': 'Maple Leafs'},{'default': 'Red Wings'},0,-2,0,0,0,0,1,15,0,0,DET,0,10:52,8479393,Noah,Gregor
667,2023020669,NYR,H,2024-01-14,0,0,{'default': 'Rangers'},{'default': 'Capitals'},0,1,0,0,0,0,3,22,0,0,WSH,0,24:02,8479323,Adam,Fox
2272,2023020669,WSH,R,2024-01-14,0,0,{'default': 'Capitals'},{'default': 'Rangers'},0,0,0,0,0,0,1,16,0,0,NYR,0,12:29,8478451,Ethan,Bear
660,2023020669,NYR,H,2024-01-14,1,0,{'default': 'Rangers'},{'default': 'Capitals'},1,2,0,0,0,0,4,18,0,0,WSH,0,18:47,8478550,Artemi,Panarin


In [247]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Vancouver,Canucks,VAN
1,Winnipeg,Jets,WPG
2,Boston,Bruins,BOS
3,Colorado,Avalanche,COL
4,Florida,Panthers,FLA
5,NY Rangers,Rangers,NYR
6,Dallas,Stars,DAL
7,Carolina,Hurricanes,CAR
8,Vegas,Golden Knights,VGK
9,Philadelphia,Flyers,PHI


In [248]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)

playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).shots.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)


playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,42,42,0.952381,0.462328,1,0.6,1,0.9
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,41,42,0.952381,0.462328,1,0.6,1,0.9
2,WPG,8475455,Brenden,Dillon,2023-12-27,0,CHI,40,42,0.952381,1.102007,1,0.6,1,0.9
3,WPG,8475455,Brenden,Dillon,2023-12-22,0,BOS,39,42,0.952381,1.102007,1,0.6,1,0.9
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,38,42,0.952381,0.462328,1,0.6,1,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,SJS,8482667,William,Eklund,2024-01-11,3,MTL,5,43,1.558140,0.817030,0,2.6,0,2.3
3468,SJS,8482667,William,Eklund,2024-01-09,2,TOR,4,43,1.558140,0.177351,0,2.6,0,2.3
3469,SJS,8482667,William,Eklund,2024-01-06,0,TOR,3,43,1.558140,1.102007,0,2.6,0,2.3
3470,SJS,8482667,William,Eklund,2024-01-04,0,WPG,2,43,1.558140,1.102007,0,2.6,0,2.3


In [249]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,42,42,0.952381,0.462328,1,0.6,1,0.9,24,24,1.291667
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,41,42,0.952381,0.462328,1,0.6,1,0.9,23,24,1.291667
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,38,42,0.952381,0.462328,1,0.6,1,0.9,22,24,1.291667
6,WPG,8475455,Brenden,Dillon,2023-12-16,1,COL,36,42,0.952381,0.462328,0,0.6,1,0.9,21,24,1.291667
7,WPG,8475455,Brenden,Dillon,2023-12-13,2,LAK,35,42,0.952381,0.177351,0,0.6,1,0.9,20,24,1.291667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23470,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,43,0.697674,0.462328,0,0.0,0,0.0,1,14,1.571429
23471,SJS,8482667,William,Eklund,2024-01-13,2,OTT,6,43,1.558140,0.177351,0,2.6,0,2.3,4,30,1.733333
23472,SJS,8482667,William,Eklund,2024-01-11,3,MTL,5,43,1.558140,0.817030,0,2.6,0,2.3,3,30,1.733333
23473,SJS,8482667,William,Eklund,2024-01-09,2,TOR,4,43,1.558140,0.177351,0,2.6,0,2.3,2,30,1.733333


In [250]:
Teams

,City,TeamName,TeamID
0,Vancouver,Canucks,VAN
1,Winnipeg,Jets,WPG
2,Boston,Bruins,BOS
3,Colorado,Avalanche,COL
4,Florida,Panthers,FLA
5,NY Rangers,Rangers,NYR
6,Dallas,Stars,DAL
7,Carolina,Hurricanes,CAR
8,Vegas,Golden Knights,VGK
9,Philadelphia,Flyers,PHI


In [260]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,SJS,BUF
1,NJD,BOS
2,VAN,CBJ
3,ANA,FLA
4,SEA,PIT
5,LAK,CAR
6,NYI,MIN
7,NSH,VGK
8,COL,MTL
9,PHI,STL


In [261]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots','LastTenAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['teamAbbrev'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='teamAbbrev', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent
0,BOS,8477956,David,Pastrnak,5.047619,5.8,6.2,4.935484,NJD
1,NJD,8481559,Jack,Hughes,3.450000,0.0,1.5,4.600000,BOS
2,COL,8477492,Nathan,MacKinnon,4.534884,5.0,4.1,4.500000,MTL
3,NSH,8476887,Filip,Forsberg,3.860465,5.8,4.1,4.088235,VGK
4,MTL,8481540,Cole,Caufield,3.666667,2.8,3.8,3.967742,COL
5,VGK,8478403,Jack,Eichel,4.023256,3.6,3.7,3.942857,NSH
6,PIT,8477404,Jake,Guentzel,3.780488,5.0,4.5,3.843750,SEA
7,PHI,8480015,Owen,Tippett,3.534884,5.0,4.1,3.787879,STL
8,BUF,8475784,Jeff,Skinner,3.116279,0.6,1.8,3.629630,SJS
9,MIN,8478493,Joel,Eriksson Ek,3.785714,5.2,4.7,3.628571,NYI


In [262]:
goalieSTARTINGCSV2023=pd.read_csv('GoalieLastYear2023Stats.csv')  
goalieSTARTINGCSV2024=pd.read_csv('GoalieThisYear2024Stats.csv')  

goalieSTARTINGCSV=pd.concat([goalieSTARTINGCSV2023, goalieSTARTINGCSV2024])
goalieSTARTINGCSV=goalieSTARTINGCSV[goalieSTARTINGCSV['gamesStarted']==1]
goalieSTARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,gamesStarted,decision,shotsAgainst,goalsAgainst,savePctg,shutouts,opponentAbbrev,pim,toi,playerId,firstName,lastName
495,2023020001,NSH,R,2023-10-10,0,0,{'default': 'Predators'},{'default': 'Lightning'},1,L,33,4,0.878788,0,TBL,0,57:16,8477424,Juuse,Saros
760,2023020003,SEA,R,2023-10-10,0,0,{'default': 'Kraken'},{'default': 'Golden Knights'},1,L,27,3,0.888889,0,VGK,0,59:37,8475831,Philipp,Grubauer
270,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,W,33,1,0.969697,0,SEA,0,60:00,8478499,Adin,Hill
1098,2023020002,CHI,R,2023-10-10,0,0,{'default': 'Blackhawks'},{'default': 'Penguins'},1,W,41,2,0.951220,0,PIT,0,60:00,8475852,Petr,Mrazek
586,2023020002,PIT,H,2023-10-10,0,0,{'default': 'Penguins'},{'default': 'Blackhawks'},1,L,35,3,0.914286,0,CHI,0,57:30,8477465,Tristan,Jarry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2023020656,NJD,R,2024-01-13,0,0,{'default': 'Devils'},{'default': 'Panthers'},1,W,37,1,0.972973,0,FLA,0,60:00,8482076,Nico,Daws
70,2023020670,TOR,H,2024-01-14,0,0,{'default': 'Maple Leafs'},{'default': 'Red Wings'},1,L,23,3,0.869565,0,DET,0,59:13,8478492,Ilya,Samsonov
126,2023020669,WSH,R,2024-01-14,0,0,{'default': 'Capitals'},{'default': 'Rangers'},1,L,31,2,0.935484,0,NYR,0,58:29,8479292,Charlie,Lindgren
77,2023020670,DET,R,2024-01-14,0,0,{'default': 'Red Wings'},{'default': 'Maple Leafs'},1,W,30,2,0.933333,0,TOR,0,59:32,8473503,James,Reimer


In [278]:
goalieStats=goalieSTARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','opponentAbbrev','gamesStarted','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['PlayerGameNumber'] = goalieStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
goalieStats['TeamGameNumber'] = goalieStats.groupby(['teamAbbrev'])['gameDate'].cumcount(ascending=False)+1
goalieStats['PlayerGP']=goalieStats.groupby(['playerId']).gamesStarted.transform(np.sum)
goalieStats['TeamGP']=goalieStats.groupby(['teamAbbrev']).gamesStarted.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['teamAbbrev']).shotsAgainst.transform(np.sum)/goalieStats['TeamGP']
goalieStats['RankAvgShotsAgainst']=goalieStats['AvgShotsAgainst'].rank(pct=True)*100

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgGoalsAgainst']=goalieStats['AvgGoalsAgainst'].rank(pct=True)*100

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgSavePctg']=goalieStats['AvgSavePctg'].rank(pct=True)*100

goalieStats['LastFiveFlag'] = np.where(goalieStats['TeamGameNumber'] >= goalieStats['TeamGP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['teamAbbrev','PlayerGP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['teamAbbrev'])['PlayerGP'].rank().astype(int)
summaryGoalieStats=summaryGoalieStats[['teamAbbrev','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','teamAbbrev':'GoalieteamAbbrev'})
summaryGoalieStats

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,WPG,1,Laurent,Brossoit,36.737089,4.929577,83.255086
9,WPG,2,Connor,Hellebuyck,36.737089,6.572770,91.627543
35,BOS,1,Linus,Ullmark,73.356808,41.510172,89.632238
53,BOS,2,Jeremy,Swayman,73.356808,22.261346,94.796557
71,VAN,2,Thatcher,Demko,66.823161,25.430360,81.298905
...,...,...,...,...,...,...,...
1075,CHI,2,Petr,Mrazek,83.255086,71.439750,43.270736
1100,CHI,1,Arvid,Soderblom,83.255086,98.474178,11.697966
1115,SJS,1,Kaapo,Kahkonen,95.109546,90.649452,24.021909
1133,SJS,2,Mackenzie,Blackwood,95.109546,92.957746,27.073552


In [280]:
full=todaysSkaters.merge(summaryGoalieStats
                         [summaryGoalieStats['GPRank']==1]
                         , left_on='Opponent', right_on='GoalieteamAbbrev', how='left').drop(columns=['GoalieteamAbbrev','GPRank'])
full=full[['teamAbbrev', 'firstName', 'lastName', 'NoOutliersAvgSOG',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName','SeasonAvgSOG', 'playerId', 
       'RankAvgGoalsAgainst', 'RankAvgSavePctg']].rename(columns={"NoOutliersAvgSOG": "ExpectedSOG"}).round(2)

full.sort_values(by=['ExpectedSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(25)

,teamAbbrev,firstName,lastName,ExpectedSOG,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg
0,BOS,David,Pastrnak,4.94,5.8,6.2,NJD,13.38,Daws,5.05,8477956,41.51,82.67
1,NJD,Jack,Hughes,4.60,0.0,1.5,BOS,73.36,Ullmark,3.45,8481559,41.51,89.63
2,COL,Nathan,MacKinnon,4.50,5.0,4.1,MTL,91.82,Primeau,4.53,8477492,81.34,64.63
3,NSH,Filip,Forsberg,4.09,5.8,4.1,VGK,51.17,Patera,3.86,8476887,99.26,40.41
4,MTL,Cole,Caufield,3.97,2.8,3.8,COL,16.12,Prosvetov,3.67,8481540,66.47,14.05
5,VGK,Jack,Eichel,3.94,3.6,3.7,NSH,28.60,Lankinen,4.02,8478403,81.34,33.72
6,PIT,Jake,Guentzel,3.84,5.0,4.5,SEA,36.74,Driedger,3.78,8477404,0.08,100.00
7,PHI,Owen,Tippett,3.79,5.0,4.1,STL,70.11,Hofer,3.53,8480015,39.12,38.26
8,BUF,Jeff,Skinner,3.63,0.6,1.8,SJS,95.11,Kahkonen,3.12,8475784,90.65,24.02
9,MIN,Joel,Eriksson Ek,3.63,5.2,4.7,NYI,98.40,Varlamov,3.79,8478493,34.90,87.56


In [281]:
margin=full
margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']

margin=margin[['teamAbbrev', 'firstName', 'lastName', 'ExpectedSOG',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName','SeasonAvgSOG', 'playerId', 
       'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

margin.sort_values(by=['lastfivemargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,firstName,lastName,ExpectedSOG,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin
0,VAN,Elias,Pettersson,2.75,5.6,3.9,CBJ,88.58,Tarasov,2.79,8480012,99.73,7.16,2.85
1,VGK,Jonathan,Marchessault,2.90,5.0,4.3,NSH,28.60,Lankinen,3.47,8476539,81.34,33.72,2.10
2,LAK,Pierre-Luc,Dubois,1.86,3.6,2.7,CAR,1.68,Andersen,1.85,8479400,29.07,0.90,1.74
3,NSH,Filip,Forsberg,4.09,5.8,4.1,VGK,51.17,Patera,3.86,8476887,99.26,40.41,1.71
4,PIT,Evgeni,Malkin,2.76,4.4,3.5,SEA,36.74,Driedger,2.73,8471215,0.08,100.00,1.64
5,MIN,Joel,Eriksson Ek,3.63,5.2,4.7,NYI,98.40,Varlamov,3.79,8478493,34.90,87.56,1.57
6,COL,Mikko,Rantanen,3.44,5.0,4.9,MTL,91.82,Primeau,3.49,8478420,81.34,64.63,1.56
7,VAN,Brock,Boeser,2.46,4.0,3.3,CBJ,88.58,Tarasov,2.70,8478444,99.73,7.16,1.54
8,PIT,Kris,Letang,2.09,3.6,2.3,SEA,36.74,Driedger,2.05,8471724,0.08,100.00,1.51
9,MIN,Marcus,Johansson,1.93,3.2,2.3,NYI,98.40,Varlamov,1.69,8475149,34.90,87.56,1.27


In [282]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df